In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


# StructuredOutputParser

1. 입출력 형식 정의: 
    - 사용자가 JSON 스키마 통해 데이터 구조, 필드 이름, 데이터 타입 등을 정의 
    - 모델 출력이 정의된 스키마에 따라 반환되도록 보장
2. 유효성 검사: 
    - 출력 데이터가 정의된 스키마에 맞지 않을 경우 오류 반환
3. 구조화된 출력:
    - 테그트 응답을 정의된 구조로 변환하여 데이터 처리 및 후속 작업을 용이하게 만듦

[Reference] https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/structured/

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [4]:
# 답변 스키마 정의
response_schemas = [
    ResponseSchema(name="answer", description="사용자 질문에 대한 답변"),
    ResponseSchema(
        name="source", description="질문에 답하기 위해 사용된 `출처`, `웹사이트주소`"
    ),
]

# 파서 정의
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [5]:
print(output_parser)

response_schemas=[ResponseSchema(name='answer', description='사용자 질문에 대한 답변', type='string'), ResponseSchema(name='source', description='질문에 답하기 위해 사용된 `출처`, `웹사이트주소`', type='string')]


In [ ]:
# 출력 지시사항
format_instructions = output_parser.get_format_instructions()

# 템플릿 정의
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [7]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 `출처`, `웹사이트주소`
}
```


In [ ]:
# 모델 정의
model = ChatOpenAI(temperature=0)

# 체인 정의
chain = prompt | model | output_parser

# 실행
chain.invoke({"question": "스페인의 대표 특산품은 무엇인가요?"})

{'answer': '스페인의 대표 특산품은 햄(진행식 햄)과 와인입니다.',
 'source': 'https://www.spain.info/ko/'}

In [11]:
# 스트리밍 출력
for a in chain.stream({"question": "스페인의 대표 특산품은 무엇인가요?"}):
    print(a)

{'answer': '스페인의 대표 특산품은 햄(진행햄)과 와인(산타 마리아 와인)입니다.', 'source': 'https://www.spain.info/ko/que-ver-en-espana/gastronomia/productos-tipicos.html'}


-----
** End of Documents **